# Ex12


Uma empresa produz televisores em 3 fábricas: **São Paulo**, **João Pessoa** e **Manaus**. Os pontos principais de revenda, com as respectivas encomendas mensais, são:

### **Encomendas nos pontos de revenda**
| Pontos de Revenda   | Encomendas (unidades) |
|---------------------|----------------------|
| **Rio de Janeiro**  | 6.000                |
| **Salvador**        | 5.000                |
| **Aracaju**         | 2.000                |
| **Maceió**          | 1.000                |
| **Recife**          | 3.000                |

### **Capacidade de produção nas fábricas**
| Fábricas    | Capacidade (unidades) |
|------------|----------------------|
| **São Paulo**   | 10.000             |
| **João Pessoa** | 5.000              |
| **Manaus**      | 6.000              |

O custo de transporte das fábricas até as revendas, para cada lote de **1.000 aparelhos**, é dado pelo quadro abaixo:

### **Custo de transporte (R$/lote de 1.000 unidades)**
|             | **Rio de Janeiro** | **Salvador** | **Aracaju** | **Maceió** | **Recife** |
|------------|-------------------------------------|-------------|------------|------------|------------|
| **São Paulo** |                 1.000              | 2.000       | 3.000      | 3.500      | 4.000      |
| **João Pessoa** |               4.000              | 2.000       | 1.500      | 1.200      | 1.000      |
| **Manaus** |                   6.000              | 4.000       | 3.500      | 3.000      | 2.000      |

### **Objetivo**
Determinar o número de unidades produzidas em cada fábrica e entregues a cada revenda, a fim de **minimizar o custo de transporte**.



## Modelagem Matemática

### **Variáveis **
- $x_{i,j}$ = número de televisores transportados da fábrica $i$ para o ponto de revenda $j$.

### **Função Objetivo**
Queremos minimizar o custo total de transporte:

$$
\min \sum_{i \in F} \sum_{j \in R} c_{i,j} \cdot x_{i,j}
$$

onde:
- $F = \{\text{São Paulo, João Pessoa, Manaus}\}$ as fábricas.
- $R = \{\text{Rio de Janeiro, Salvador, Aracaju, Maceió, Recife}\}$ os pontos de revenda.

### **Restrições**
1. **Capacidade de produção das fábricas**:
   Cada fábrica só pode fornecer até sua capacidade máxima:

   $$
   \sum_{j \in R} x_{\text{SP},j} \leq 10.000
   $$

   $$
   \sum_{j \in R} x_{\text{JP},j} \leq 5.000
   $$

   $$
   \sum_{j \in R} x_{\text{MA},j} \leq 6.000
   $$

2. **Atendimento da demanda dos pontos de revenda**:
   Cada ponto de revenda precisa receber exatamente o número solicitado de televisores:

   $$
   \sum_{i \in F} x_{i,\text{RJ}} = 6.000
   $$

   $$
   \sum_{i \in F} x_{i,\text{Salvador}} = 5.000
   $$

   $$
   \sum_{i \in F} x_{i,\text{Aracaju}} = 2.000
   $$

   $$
   \sum_{i \in F} x_{i,\text{Maceió}} = 1.000
   $$

   $$
   \sum_{i \in F} x_{i,\text{Recife}} = 3.000
   $$

3. **Não negatividade**:
   Todas as variáveis de transporte devem ser não negativas:

   $$
   x_{i,j} \geq 0, \quad \forall i \in F, j \in R
   $$



In [23]:
import pyomo.environ as pyo

In [24]:
# Criação do modelo
model = pyo.ConcreteModel()

In [25]:

# Conjuntos
model.F = pyo.Set(initialize=["SP", "JP", "MA"])  
model.R = pyo.Set(initialize=["RJ", "Salvador", "Aracaju", "Maceió", "Recife"]) 


In [26]:
# Capacidade de produção nas fábricas
capacidade_producao = {"SP": 10000, "JP": 5000, "MA": 6000}
model.cap = pyo.Param(model.F, initialize=capacidade_producao)

# Demanda dos pontos de revenda
demanda_revenda = {"RJ": 6000, "Salvador": 5000, "Aracaju": 2000, "Maceió": 1000, "Recife": 3000}
model.demanda = pyo.Param(model.R, initialize=demanda_revenda)

# Custos de transporte (R$/unidade)
custo_transporte = {
    ("SP", "RJ"): 1.00, ("SP", "Salvador"): 2.00, ("SP", "Aracaju"): 3.00, ("SP", "Maceió"): 3.50, ("SP", "Recife"): 4.00,
    ("JP", "RJ"): 4.00, ("JP", "Salvador"): 2.00, ("JP", "Aracaju"): 1.50, ("JP", "Maceió"): 1.20, ("JP", "Recife"): 1.00,
    ("MA", "RJ"): 6.00, ("MA", "Salvador"): 4.00, ("MA", "Aracaju"): 3.50, ("MA", "Maceió"): 3.00, ("MA", "Recife"): 2.00
}
model.custo = pyo.Param(model.F, model.R, initialize=custo_transporte)

In [27]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.F, model.R, domain=pyo.NonNegativeIntegers)

In [28]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[i, j] * model.custo[i, j] for i in model.F for j in model.R)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [29]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Capacidade de produção nas fábricas
def capacidade(model, i):
    return sum(model.x[i, j] for j in model.R) <= model.cap[i]

model.Capacidade = pyo.Constraint(model.F, rule=capacidade)

# 2. Atendimento da demanda dos pontos de revenda
def atendimento_demanda(model, j):
    return sum(model.x[i, j] for i in model.F) == model.demanda[j]
model.AtendimentoDemanda = pyo.Constraint(model.R, rule=atendimento_demanda)



In [30]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex12.lp", io_options={"symbolic_solver_labels": True})

('ex12.lp', 2653227746928)

In [31]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 8 rows; 15 cols; 30 nonzeros; 15 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 6e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 1e+04]
Presolving model
8 rows, 15 cols, 30 nonzeros  0s
8 rows, 15 cols, 30 nonzeros  0s
Objective function is integral with scale 10

Solving MIP model with:
   8 rows
   15 cols (0 binary, 15 integer, 0 implied int., 0 continuous)
   30 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0        

In [32]:
for i in model.F:
    for j in model.R:
        val = pyo.value(model.x[i, j])
        if val > 0:  
            print(f"Quantidade transportada da fábrica {i} para {j}: {val:.2f}")

# Exibindo o valor da função objetivo
print("\nValor da Função Objetivo: R$", formatted)

Quantidade transportada da fábrica SP para RJ: 6000.00
Quantidade transportada da fábrica SP para Salvador: 4000.00
Quantidade transportada da fábrica JP para Salvador: 1000.00
Quantidade transportada da fábrica JP para Aracaju: 2000.00
Quantidade transportada da fábrica JP para Maceió: 1000.00
Quantidade transportada da fábrica JP para Recife: 1000.00
Quantidade transportada da fábrica MA para Recife: 2000.00

Valor da Função Objetivo: R$ 25.200,00
